In [ ]:
# problem : need to find vibe shift theta of unitary on one of its 'chill' eigenvector
#
# 1.bring counting/extra bits and initialize to zero state
#  Initial stage : ∣0⟩⊗t∣ψ⟩
#
# 2.Applying hadamard gate to each counting bits
#
# 3.Apllying controlled version of unitary operator to eigenvectors
# If the first counting qubit is ∣1⟩, we apply U 
#  If the first counting qubit is ∣1⟩, we apply U raised to 2 raised to 0 = U once to ∣ψ⟩
# If the second counting qubit is ∣1⟩, we apply U raised to 2 raised to 1=U raised to 2 once to ∣ψ⟩.
# If the third counting qubit is ∣1⟩, we apply U raised to 2 raised to 2=U raised to 4 once to ∣ψ⟩. ... and so on, up to the t-th counting qubit controlling U raised to 2 raised to t−1

# 4. Decoding the theta as its in scramble form using IQFT a reverse freqency analysis operations


In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.circuit.library import QFT

In [9]:
def qpe(uni_op,eigen,num_qubits):
    
    # Intialized the Quantum circuits
    qpe_circuit=QuantumCircuit(num_qubits+eigen.num_qubits,num_qubits)
    counting_qubits=qpe_circuit.qubits[:num_qubits]
    eigenvector_qubits=qpe_circuit.qubits[num_qubits:]
    classical_bits=qpe_circuit.clbits
    
    # prepare the eigenvectors
    qpe_circuit.compose(eigen,eigenvector_qubits,inplace=True)
    
    #  apply Hadamard Gate to counting bits
    qpe_circuit.h(counting_qubits)
    
    # apply controlles power of unitary op
    for i in range(num_qubits):
        controlled_u=uni_op.control(1)
        power=2**i
        for _ in range(power):
            qpe_circuit.append(controlled_u,[counting_qubits[i]]+eigenvector_qubits)
    
    # Aplly inverse Quantum Fourier Analysis
    iqft= QFT(num_qubits,inverse=True)
    qpe_circuit.compose(iqft,counting_qubits,inplace=True)
    
    # Measure the counting qubits
    qpe_circuit.measure(counting_qubits,classical_bits)
    
    return qpe_circuit

In [ ]:
def counts(circuit):
    simulator=AerSimulator()
    compiled_circuit=transpile(circuit,simulator)
    job=simulator.run(compiled_circuit,shots=1024)
    result=job.result()
    counts=result.get_counts()
    return counts

In [18]:
# Example : estimating phase of Z gate
count_bits_num=5

# Unitary operation
unitary_op=QuantumCircuit(1,name='U')
unitary_op.z(0)

# EigenVector
eigenvec = QuantumCircuit(1,name='psi')
eigenvec.x(0)

qpe_circuit=qpe(unitary_op,eigenvec,count_bits_num)



In [ ]:
# simulate the circuit
result=counts(qpe_circuit)
print("Measurement : ",result)

#Result interpretation
estimated_phase=0
max=0
for output,count in result.items():
    if count > max:
        max=count
        measured_int=int(output,2)
        estimated_phase=measured_int/(2**count_bits_num)

print(f"Estimated phase (theta): {estimated_phase}")
print(f"True phase (theta for Z on |1>): 0.5")

Measurement :  {'10000': 10000}
Estimated phase (theta): 0.5
True phase (theta for Z on |1>): 0.5


In [21]:
# Example : estimating phase of Z gate
count_bits_num=4

# Unitary operation
unitary_op=QuantumCircuit(1,name='U')
unitary_op.t(0)

# EigenVector
eigenvec = QuantumCircuit(1,name='psi')
eigenvec.x(0)

qpe_circuit=qpe(unitary_op,eigenvec,count_bits_num)



In [23]:
# simulate the circuit
result=counts(qpe_circuit)
print("Measurement : ",result)

#Result interpretation
estimated_phase=0
max=0
for output,count in result.items():
    if count > max:
        max=count
        measured_int=int(output,2)
        estimated_phase=measured_int/(2**count_bits_num)
        
print(f"Estimated phase (theta): {estimated_phase}")
print(f"True phase (theta for T on |1>): 0.125")

Measurement :  {'0010': 10000}
Estimated phase (theta): 0.125
True phase (theta for T on |1>): 0.125
